In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
from xgcm import Grid
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
import regionate as reg
import sectionate as sec

In [5]:
import pickle
BasinsFile = "/work/hfd/datasets/regionate/pickled_regions/PJ2010_region_gridded_CM4p125"
with open(BasinsFile, 'rb') as pickle_file:
    region = pickle.load(pickle_file)

In [6]:
ds = xr.open_mfdataset("/work/hfd/CM4-highres-WMT/data/abyssal_water_mass_transports_*.nc")
grid = Grid(ds, coords={'X': {'center': 'xh', 'outer': 'xq'},
                        'Y': {'center': 'yh', 'outer': 'yq'}}, periodic=['X'])
sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p125/mosaic_c192_om4p125_bedmachine_v20210310_hydrographyKDunne20210614_unpacked/ocean_hgrid.nc")
zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_rho2/"
og = xr.open_dataset(f"{zdiag_path}ocean_annual_rho2.static.nc")
og = og.assign_coords({'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
                                       'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"]),
                                       'geolon_u': xr.DataArray(sg['x'][1::2,0::2].data, dims=["yh", "xq"]),
                                       'geolat_u': xr.DataArray(sg['y'][1::2,0::2].data, dims=["yh", "xq"]),
                                       'geolon_v': xr.DataArray(sg['x'][0::2,1::2].data, dims=["yq", "xh"]),
                                       'geolat_v': xr.DataArray(sg['y'][0::2,1::2].data, dims=["yq", "xh"]),
                                       'geolon_c': xr.DataArray(sg['x'][0::2,0::2].data, dims=["yq", "xq"]),
                                       'geolat_c': xr.DataArray(sg['y'][0::2,0::2].data, dims=["yq", "xq"])})
ds = ds.assign_coords({
    'dxCv': xr.DataArray(og['dxCv'].transpose('xh', 'yq').values, dims=('xh', 'yq',)),
    'dyCu': xr.DataArray(og['dyCu'].transpose('xq', 'yh').values, dims=('xq', 'yh',))
}) # add velocity face widths to calculate distances along the section
symmetric = og.geolon.shape == og.geolon_c.shape

rho0 = 1035.

In [ ]:
print("Computing interbasin transports: ", end="")
for bname, b in region.Basins.items():
    print(bname, end=", ")
    convergent_transport = sec.convergent_transport(ds.isel(time=-1), b.i, b.j, symmetric, layer=None, interface=None)['conv_mass_transport'].compute()/rho0*1e-6
    b.psi = convergent_transport
    b.psi_net = convergent_transport.sum('sect')
    b_overlaps = {pair:o for pair,o in region.overlaps.items() if b.name in pair}
    b.psi_exchange = {}
    for pair, o in b_overlaps.items():
        b.psi_exchange[pair] = convergent_transport.isel(sect=reg.flatten_odict(o[b.name])).sum('sect')

Computing interbasin transports: 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 2, 30, 31, 32, 33, 3, 4, 5, 6, 7, 8, 9, 35, 

In [ ]:
regionFile_CM4p125_moc = "/work/hfd/datasets/regionate/pickled_regions/PJ2010_region_gridded_CM4p125_moc"
with open(regionFile_CM4p125_moc, 'wb') as pickle_file:
    pickle.dump(region, pickle_file)